# Boid Simulation 1.5
- I am eager to view a swarm in 3d after having figured out how to use open 3d to do so, my old code can be modified to precompute the positions then playback the *frames*
- Code by Michael Naguib 8/8/19

### Imports

In [2]:
#imports
from tkinter import *
from BoidSwarm import BoidSwarm
from Vector import Vector
import time
import tqdm
import math
import copy
import numpy as np

### Run a similation in window live time

In [2]:
    HEIGHT = 600
    WIDTH = round(HEIGHT * 1.6180339) # preserve a nice aspect ratio by using phi= THE GOLDEN RATIO (one of my favorite numbers)
    DEPTH = HEIGHT
    QUANTITY=900

    #=== Setup the Graphics
    screen_dim = Vector([WIDTH,HEIGHT,DEPTH])
    root = Tk()
    root.geometry('%dx%d+%d+%d' % (WIDTH, HEIGHT, (root.winfo_screenwidth() - WIDTH) / 2, (root.winfo_screenheight() - HEIGHT) / 2))
    root.bind_all('<Escape>', lambda event: event.widget.quit())
    graph = Canvas(root, width=WIDTH, height=HEIGHT, background='white')
    graph.pack()

    #=== Setup the swarm
    mySwarm = BoidSwarm(QUANTITY,screen_dim)
    mySwarm.setup()
    def work():
        mySwarm.update_boid_positions()  # Run the Boid Calculation
    #=== Enter the main loop
    while True:
        graph.delete(ALL)#                  Clear the screen
        work()
        mySwarm.draw_swarm(graph)#          Draw the swarm
        graph.update()#                     Update the screen

TclError: invalid command name ".!canvas"

### Simulation Setup For a Precomputed Simulation

In [3]:
#=== Settings
HEIGHT = 600
WIDTH = round(HEIGHT * 1.6180339) # preserve a nice aspect ratio by using phi= THE GOLDEN RATIO (one of my favorite numbers)
DEPTH = HEIGHT
QUANTITY=500
BOIDCOLOR='black'
BOIDSCALE=5
timeDelay = 0.02 # 0.5=half second ... used to space out each frame in the playback
#Number of desired frames to compute...
K = int(math.pow(10,2)*2)
runtime = int(100*K*timeDelay/60.0)/100
print("The Similation will last about: {0} min".format(str(runtime)))

The Similation will last about: 0.06 min


In [4]:
#=== Setup the Graphics
screen_dim = Vector([WIDTH,HEIGHT,DEPTH])
root = Tk()
root.geometry('%dx%d+%d+%d' % (WIDTH, HEIGHT, (root.winfo_screenwidth() - WIDTH) / 2, (root.winfo_screenheight() - HEIGHT) / 2))
root.bind_all('<Escape>', lambda event: event.widget.quit())
graph = Canvas(root, width=WIDTH, height=HEIGHT, background='white')
graph.pack()

#=== Setup the swarm
mySwarm = BoidSwarm(QUANTITY,screen_dim)
mySwarm.setup()

### Draw Positions Func

In [5]:
#A modified version of the boid draw function
def drawPos(tkinter_canvas,posLst,scale,color):
    '''
    :description: takes a tkinter canvas and draws a representation of the boid to the canvas (only does 2d...)
    :param tkinter_canvas: a tkinter canvas object
    '''
    #draw a circle whose center is at the current position of the boid and with a radius of the scale factor..
    d2Cord = (posLst[0]+ scale,
              posLst[1]+ scale,
              posLst[0]- scale,
              posLst[1]- scale)
    #draw the boid ... note using syntatic sugar for position update
    tkinter_canvas.create_oval(d2Cord,fill=color)

### Precompute Running the simulation for K frames...

In [6]:
#Each frame holds a list of position vectors
positionFrameHistory=[]#very big list...

#Run the simulation
for i in tqdm.tqdm(range(0,K)):
    #Calculate the Frame
    mySwarm.update_boid_positions()  # Run the Boid Calculation
    posData = [copy.deepcopy(agent.pos.components) for agent in mySwarm.boid_list]#only take components
    positionFrameHistory.append(posData)

100%|████████████████████████████████████████| 200/200 [03:40<00:00,  1.01s/it]


### Playback the Simulation 2D

In [6]:
#=== Enter the main loop
for frame in positionFrameHistory:
    graph.delete(ALL)#                  Clear the screen
    mySwarm.update_boid_positions()  # Run the Boid Calculation
    for posLst in frame:
        drawPos(graph,posLst,BOIDSCALE,BOIDCOLOR)
    graph.update()#                     Update the screen

### Play Back the simulation 3d

In [7]:
import open3d
import time
import numpy as np
pcd = open3d.PointCloud()

vis = open3d.Visualizer()
vis.create_window()
vis.add_geometry(pcd)

#Setup the initial frame
pcd.points = open3d.Vector3dVector(positionFrameHistory[0])
pcd.colors = open3d.Vector3dVector([[0,0,0] for i in range(0,len(positionFrameHistory[0]))])

render_option = vis.get_render_option()
render_option.point_size = 5#0.01
#precompute
for i in range(1,len(positionFrameHistory)):
    positionFrameHistory[i]=open3d.Vector3dVector(positionFrameHistory[i])
#render_option.point_color_option = open3d.visualization.PointColorOption.Normal
to_reset_view_point = True
for i in range(1,len(positionFrameHistory)):    
    pcd.points = positionFrameHistory[i]
    vis.update_geometry()
    if to_reset_view_point:
        vis.reset_view_point(True)
        to_reset_view_point = False
    vis.poll_events()
    vis.update_renderer()
    time.sleep(timeDelay)# could be interpreted as frames per second assuming the calculation is instant and retrieval is as well

vis.destroy_window()

In [ ]:
#